In [1]:
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
def aceptar_cookies(año, liga, jornada):
    driver_dir = 'chrome_driver/chromedriver.exe'
    driver=webdriver.Chrome(driver_dir)
    driver.get("https://www.resultados-futbol.com/"+liga+str(año)+"/grupo1/jornada"+str(jornada))
    accept_cookies = driver.find_elements_by_xpath('//button[@class="sc-ifAKCX hYNOwJ"]')
    
    try:
        for button in accept_cookies:
            if button.text == "ACEPTO":
                relevant_button = button
                relevant_button.click()
    except:
        pass
    finally:
        return driver

In [15]:
def extraer_datos_jornada(soup):
    '''
    DOCSTRING
    '''
    if soup.find("table", {"id": 'tabla2'}):
        table_soup = soup.find("table", {"id": 'tabla2'}).find('tbody').find_all('tr')
    else:
        return None
    num_equipos = len(table_soup)

    Posicion = [table_soup[i].find('th').text for i in range(num_equipos)]
    Equipo = [table_soup[i].find('td', {'class':'equipo'}).find('a').text for i in range(num_equipos)]
    Puntos = [table_soup[i].find('td', {'class':'pts'}).text for i in range(num_equipos)]
    Jornada = [table_soup[i].find('td', {'class':'pj'}).text for i in range(num_equipos)]
    Ganados = [table_soup[i].find('td', {'class':'win'}).text for i in range(num_equipos)]
    Empatados = [table_soup[i].find('td', {'class':'draw'}).text for i in range(num_equipos)]
    Perdidos = [table_soup[i].find('td', {'class':'lose'}).text for i in range(num_equipos)]
    Goles_a_favor= [table_soup[i].find('td', {'class':'f'}).text for i in range(num_equipos)]
    Goles_en_contra= [table_soup[i].find('td', {'class':'c'}).text for i in range(num_equipos)]

    tabla_jornada = [Posicion, Equipo, Puntos, Jornada, Ganados, Empatados, Perdidos, Goles_a_favor, Goles_en_contra]

    # Nos aseguramos que ningun dato se ha saltado, y que hemos encontrado todos
    if len(set([len(i) for i in tabla_jornada])) == 1:
        return tabla_jornada
    else:
        return [None] * len(tabla_jornada)

In [16]:
def extraer_datos_partido(soup):
    '''
    DOCSTRING
    '''
    regex = re.compile('vevent')
    if soup.find("table", {"id": 'tabla1'}):
        table_soup = soup.find("table", {"id": 'tabla1'}).find('tbody').find_all("tr", {"class" : regex})
    else:
        return None
    
    num_partidos = len(table_soup)
    Equipo_Local = [table_soup[i].find('td', {'class':'equipo1'}).find_all('a')[1].text for i in range(num_partidos)]
    Equipo_Visitante = [table_soup[i].find('td', {'class':'equipo2'}).find_all('a')[1].text for i in range(num_partidos)]
    Resultado = [table_soup[i].find('td', {'class':'rstd'}).find('a', {'class':'url'}).find('span').text for i in range(num_partidos)]
    Fecha = [table_soup[i].find('td', {'class':'fecha'}).text for i in range(num_partidos)]

    tabla_partido = [Equipo_Local, Equipo_Visitante, Resultado, Fecha]

    # Nos aseguramos que ningun dato se ha saltado, y que hemos encontrado todos
    if len(set([len(i) for i in tabla_partido])) == 1:
        return tabla_partido
    else:
        return [None] * len(tabla_partido)

In [5]:
def extraer_num_jornadas(soup):
    jornadas_soup = soup.find("div", {"id":'desplega_jornadas', "class":'hidden'}).find_all('li')
    return len(jornadas_soup)

In [6]:
def extraccion_datos(Ano1, Ano2, *Ligas):
    '''
    DOCSTRING
    '''
    for liga in Ligas:
        for año in range(Ano1, Ano2):
            print(f'Extrayendo datos del año {año}') 
            driver = aceptar_cookies(año, liga, 1) # Accedemos simplemente a la primera jornada de ese año para comprobar cuantas jornadas hubo ese año
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
            num_jornadas = extraer_num_jornadas(soup)
            driver.quit()
            for jornada in range(1, num_jornadas + 1):
                print(f'\tExtrayendo jornada numero {jornada} del año {año}') 
                driver = aceptar_cookies(año, liga, jornada)
                page = driver.page_source
                soup = BeautifulSoup(page, 'html.parser')
                tabla_jornada = extraer_datos_jornada(soup)
                tabla_partido = extraer_datos_partido(soup)

                if tabla_jornada == None or tabla_partido == None:
                    print(f'------------------------------------------------------')
                    print(f'!!! La jornada {jornada} no existe en el año {año} !!!')
                    print(f'!!!            Saltando al siguiente año           !!!')
                    print(f'------------------------------------------------------')
                    break
                
                for i, key in enumerate(list_jornadas[:-2]):
                    dict_jornadas[key].extend(tabla_jornada[i])

                dict_jornadas['Año'].extend([año] * len(tabla_jornada[0]))
                dict_jornadas['Liga'].extend([liga] * len(tabla_jornada[0]))

                for i, key in enumerate(list_partidos[:-3]):
                    dict_partidos[key].extend(tabla_partido[i])

                dict_partidos['Año_partido'].extend([año] * len(tabla_partido[0]))
                dict_partidos['Jornada_partido'].extend([jornada] * len(tabla_partido[0]))
                dict_partidos['Liga_partido'].extend([liga] * len(tabla_partido[0]))
               
                print(f'\tDatos extraídos, quedan {num_jornadas - jornada} jornadas restantes este año')
                driver.quit()
            print(f'Datos del año {año} extraídos, quedan {Ano2 - año - 1} años restantes') 

In [7]:
list_jornadas = ['Posicion', 'Equipo', 'Puntos', 'Jornada', 'Ganados', 'Empatados', 'Perdidos', 'Goles_a_favor', 'Goles_en_contra', 'Año', 'Liga'] 
list_partidos = ['Equipo_Local', 'Equipo_Visitante', 'Resultado', 'Fecha', 'Año_partido', 'Jornada_partido', 'Liga_partido']
dict_jornadas = {x:[] for x in list_jornadas}
dict_partidos = {x:[] for x in list_partidos}

In [11]:
extraccion_datos(2003, 2005, 'premier', 'championship')

rayendo jornada numero 24 del año 2004
	Datos extraídos, quedan 14 jornadas restantes este año
	Extrayendo jornada numero 25 del año 2004
	Datos extraídos, quedan 13 jornadas restantes este año
	Extrayendo jornada numero 26 del año 2004
	Datos extraídos, quedan 12 jornadas restantes este año
	Extrayendo jornada numero 27 del año 2004
	Datos extraídos, quedan 11 jornadas restantes este año
	Extrayendo jornada numero 28 del año 2004
	Datos extraídos, quedan 10 jornadas restantes este año
	Extrayendo jornada numero 29 del año 2004
	Datos extraídos, quedan 9 jornadas restantes este año
	Extrayendo jornada numero 30 del año 2004
	Datos extraídos, quedan 8 jornadas restantes este año
	Extrayendo jornada numero 31 del año 2004
	Datos extraídos, quedan 7 jornadas restantes este año
	Extrayendo jornada numero 32 del año 2004
	Datos extraídos, quedan 6 jornadas restantes este año
	Extrayendo jornada numero 33 del año 2004
	Datos extraídos, quedan 5 jornadas restantes este año
	Extrayendo jornada

AttributeError: 'NoneType' object has no attribute 'find'

In [9]:
Data_Tabla=pd.DataFrame(dict_jornadas).drop_duplicates()

In [10]:
Data_Tabla.tail(200)

,Posicion,Equipo,Puntos,Jornada,Ganados,Empatados,Perdidos,Goles_a_favor,Goles_en_contra,Año,Liga
8756,1,Man. Utd,57,26,17,6,3,59,33,2000,premier
8757,2,Leeds United,51,26,16,3,7,39,29,2000,premier
8758,3,Arsenal,47,26,14,5,7,46,27,2000,premier
8759,4,Liverpool,47,26,14,5,7,38,23,2000,premier
8760,5,Chelsea,46,26,13,7,6,37,24,2000,premier
...,...,...,...,...,...,...,...,...,...,...,...
8951,16,Derby County,36,35,9,9,17,43,52,2000,premier
8952,17,Wimbledon FC,32,35,7,11,17,44,67,2000,premier
8953,18,Bradford City,30,35,7,9,19,34,65,2000,premier
8954,19,Sheffield,28,35,7,7,21,33,63,2000,premier


In [35]:
Data_Tabla.to_excel("DDBB_Tabla_España_"+str(1990)+"_"+str(1996)+".xlsx")

In [ ]:
Data_Tabla=pd.DataFrame([Posicion,Equipo,Puntos,Año,Jornada,Ganados,Empatados,Perdidos,Goles_a_favor,Goles_en_contra,Liga]).transpose()
Data_Tabla.columns=['Posicion','Equipo','Puntos','Año','Jornada','Ganados','Empatados','Perdidos','Goles_a_favor','Goles_en_contra','Liga']
Data_Tabla.to_excel("DDBB_Tabla_España_"+str(Ano1)+"_"+str(Ano2)+".xlsx") 

Data=pd.DataFrame([Equipo_Local,Equipo_Visitante,Resultado,Año_,Jornada_,Fecha,Liga_]).transpose()
Data.columns=['Equipo_Local','Equipo_Visitante','Resultado','Año','Jornada','Fecha','Liga']
Data.to_excel("DDBB_Resultados_España_"+str(Ano1)+"_"+str(Ano2)+".xlsx")  